<a href="https://colab.research.google.com/github/PavleSavic/ML_project2023/blob/main/03Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
RANDOM_STATE=1234
np.random.seed(RANDOM_STATE)

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from transformers import BertTokenizer, TFAutoModel

In [19]:
data = pd.read_csv('dataset/IMDB_dataset.csv')

In [20]:
data.sentiment.replace("positive", 1, inplace = True)
data.sentiment.replace("negative", 0, inplace = True)

In [21]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [23]:
train_ds, test_ds = train_test_split(data, test_size=0.33, random_state=RANDOM_STATE,
                                     shuffle=True, stratify=data['sentiment'])

In [24]:
train_ds.shape, test_ds.shape

((33500, 2), (16500, 2))

## Preprocesiranje

In [25]:
seq_len = 512
num_samples = len(train_ds)

# inicijalizacija:
# token ids
Xids = np.zeros((num_samples, seq_len))
# attention mask
Xmask = np.zeros((num_samples, seq_len))

Xids.shape, Xmask.shape

((33500, 512), (33500, 512))

In [26]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [27]:
for i, review in enumerate(train_ds['review']):
  # odseca sve nakon 512. tokena, ako ih ima
  tokens = tokenizer.encode_plus(review, max_length=seq_len, truncation=True,
                                 padding='max_length', add_special_tokens=True,
                                 return_tensors='tf')
  Xids[i, :] = tokens['input_ids']
  Xmask[i, :] = tokens['attention_mask']
# special tokens:
# [CLS] - start of sequence, [SEP] - separator, [PAD] - padding token

In [28]:
Xids

array([[  101.,   128.,  1191., ...,     0.,     0.,     0.],
       [  101.,  1109.,  1642., ...,     0.,     0.,     0.],
       [  101.,  1448.,  1104., ...,     0.,     0.,     0.],
       ...,
       [  101.,  1188.,  1273., ...,     0.,     0.,     0.],
       [  101., 12008., 27788., ...,     0.,     0.,     0.],
       [  101.,   138.,  1662., ...,   133.,  9304.,   102.]])

In [29]:
# 1 - bert racuna attention za taj token, 0 - ne racuna
Xmask

array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 1., 1., 1.]])

In [33]:
labels = train_ds['sentiment']
Xids.shape, Xmask.shape, labels.shape

((33500, 512), (33500, 512), (33500,))

In [35]:
dataset = tf.data.Dataset.from_tensor_slices((Xids, Xmask, labels))
dataset.take(1)

<_TakeDataset element_spec=(TensorSpec(shape=(512,), dtype=tf.float64, name=None), TensorSpec(shape=(512,), dtype=tf.float64, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [39]:
Xids[0, :].shape, Xmask[0, :].shape

((512,), (512,))

In [40]:
# reformat
def map_func(input_ids, masks, labels):
  return {'input_ids': input_ids, 'attention_mask': masks}, labels

In [41]:
dataset = dataset.map(map_func)

In [43]:
dataset.take(1)

<_TakeDataset element_spec=({'input_ids': TensorSpec(shape=(512,), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(512,), dtype=tf.float64, name=None)}, TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [44]:
batch_size = 16
dataset = dataset.shuffle(RANDOM_STATE).batch(batch_size, drop_remainder=True)

# za svaki tenzor sada imamo 16 instanci
dataset.take(1)

<_TakeDataset element_spec=({'input_ids': TensorSpec(shape=(16, 512), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(16, 512), dtype=tf.float64, name=None)}, TensorSpec(shape=(16,), dtype=tf.int64, name=None))>

In [61]:
# podela na trening i validacioni
split = 0.8
size = int((num_samples / batch_size))

train_dataset = dataset.take(size)
val_dataset = dataset.skip(size)

## Konfigurisanje modela

In [46]:
# glavni model
bert = TFAutoModel.from_pretrained('bert-base-cased')

bert.summary()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Model: "tf_bert_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  108310272 
                                                                 
Total params: 108310272 (413.17 MB)
Trainable params: 108310272 (413.17 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [50]:
# ulazni slojevi
input_ids = Input(shape=(seq_len,), name='input_ids', dtype='int32')
mask = Input(shape=(seq_len,), name='attention_mask', dtype='int32')

embeddings = bert.bert(input_ids, attention_mask=mask)[1]

x = Dense(units=1024, activation='relu') (embeddings)
# izlazni sloj - verovatnoca pozitivne klase
y = Dense(units=1, activation='sigmoid') (x)

In [52]:
model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

# zamrznuti parametri bert sloja (ograniceni resursi)
model.layers[2].trainable = False

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 512)]                0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, 512)]                0         []                            
 )                                                                                                
                                                                                                  
 bert (TFBertMainLayer)      TFBaseModelOutputWithPooli   1083102   ['input_ids[0][0]',           
                             ngAndCrossAttentions(last_   72         'attention_mask[0][0]']      
                             hidden_state=(None, 512, 7                                     

## Treniranje modela

In [59]:
model.compile(optimizer=Adam(learning_rate=1e-5, weight_decay=1e-6),
              loss=BinaryCrossentropy(), metrics=['accuracy'])

In [62]:
history = model.fit(train_dataset, validation_data=val_dataset, epochs=3, verbose=1)

Epoch 1/3
2093/2093 [==============================] - 1459s 692ms/step - loss: 0.6414 - accuracy: 0.6369
Epoch 2/3
2093/2093 [==============================] - 1459s 697ms/step - loss: 0.5980 - accuracy: 0.6928
Epoch 3/3
2093/2093 [==============================] - 1461s 698ms/step - loss: 0.5764 - accuracy: 0.7105


## Evaluacija modela

In [71]:
test_ds.head()

,review,sentiment
14746,The first film was quite hip and had amusing m...,0
28039,This first-rate western tale of the gold rush ...,1
21289,"In a near future, the ordinary man above any s...",1
12020,"A fine Martino outing, this is a spirited and ...",1
14208,ELEPHANT WALK may not be the acme of literatur...,1
